In [10]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import warnings
import os

warnings.filterwarnings("ignore")

# Configuration du WebDriver
driver = webdriver.Chrome()
url = "https://www.trustpilot.com/categories/atm"
driver.get(url)
wait = WebDriverWait(driver, 5)
time.sleep(2)


In [11]:
try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler'))
    )
    button.click()
    print("Bouton cliqué avec succès.")
except Exception as e:
    print("Impossible de cliquer sur le bouton :", e)


Bouton cliqué avec succès.


In [23]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialisation des données
liens = []  # Stockera tous les liens d'entreprises
score_etoile = []  # Stockera les pourcentages d'avis 5 étoiles
commentaires_par_agence = []  # Stockera les commentaires ou informations additionnelles

# Étape 1 : Extraire les liens des pages et des entreprises
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
)

# Extraire les liens des entreprises sur la page actuelle
for element in elements:
    link = element.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')
    liens.append(href)

# Naviguer vers la page suivante
try:
    next_page_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
    next_page_link = next_page_button.get_attribute('href')

    # Ouvrir la page suivante dans un nouvel onglet
    driver.execute_script(f"window.open('{next_page_link}', '_blank');")
    driver.switch_to.window(driver.window_handles[-1])

    # Extraire les liens supplémentaires des sections spécifiques
    elements_1 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[4]/a")
    for element in elements_1:
        href = element.get_attribute('href')
        liens.append(href)

    elements_2 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[5]/a")
    for element in elements_2:
        href = element.get_attribute('href')
        liens.append(href)

    # Fermer l'onglet de la page suivante et revenir à l'onglet principal
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
except Exception:
    print("Pas de bouton de page suivante trouvé.")

# Étape 2 : Extraire les informations détaillées des entreprises
for lien in liens:
    try:
        # Ouvrir le lien dans un nouvel onglet
        driver.execute_script(f"window.open('{lien}', '_blank');")
        driver.switch_to.window(driver.window_handles[-1])

        # Extraire le score des avis 5 étoiles
        try:
            score_5_etoile_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/main/div/div[4]/section/div[2]/div[2]/label[1]/p[2]')
            score_5_etoile = score_5_etoile_element.text.replace('%', '').strip()
            score_etoile.append(float(score_5_etoile) if score_5_etoile.isdigit() else 'NA')
        except Exception:
            score_etoile.append('NA')

        # Extraire le titre ou d'autres informations importantes
        try:
            titre = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/main/div/div[3]/div[2]/div/div/div/section/div[1]/div[2]/h1/span[1]'))
            )
            titre_text = titre.text.strip()
            commentaires_par_agence.append(titre_text)
        except Exception:
            commentaires_par_agence.append('NA')

        # Fermer l'onglet et revenir à l'onglet principal
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    except Exception as e:
        print(f"Erreur lors de l'extraction des détails pour le lien : {lien}. Erreur : {e}")
        score_etoile.append('NA')
        commentaires_par_agence.append('NA')

# Étape 3 : Afficher les résultats ou les intégrer dans un DataFrame
data = {
    'Lien': liens,
    'five_Star_Percentage': score_etoile,
    'Commentaire': commentaires_par_agence
}

df_details = pd.DataFrame(data)

# Afficher les premières lignes pour vérification
print(df_details.head())

                                                Lien  five_Star_Percentage  \
0         https://www.trustpilot.com/review/egcu.org                  95.0   
1  https://www.trustpilot.com/review/libertyfirst...                  92.0   
2  https://www.trustpilot.com/review/cryptodispen...                  65.0   
3       https://www.trustpilot.com/review/vapetm.com                 100.0   
4  https://www.trustpilot.com/review/anytime-capi...                  93.0   

                  Commentaire  
0      Evergreen Credit Union  
1  Liberty First Credit Union  
2           Crypto Dispensers  
3                      VapeTM  
4             Anytime Capital  


In [3]:
from selenium.webdriver.common.by import By
import pandas as pd

# Fonction pour extraire les informations de div_elements sur une page
def extract_div_elements(driver):
    div_elements = driver.find_elements(By.XPATH, '//div[@class="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"]')
    result_list = []  # Liste pour stocker les résultats

    for div in div_elements:
        try:
            # Localiser les sous-éléments dans chaque carte
            elements = div.find_element(By.CLASS_NAME, 'styles_footer__DoJVj')
            elements_2 = elements.find_element(By.CLASS_NAME, 'styles_wrapper___E6__')
            span_elements = elements_2.find_elements(By.TAG_NAME, 'span')

            # Concaténer le texte de tous les <span>
            concatenated_text = ''
            for span in span_elements:
                concatenated_text += span.text + '·'  # Ajout du séparateur explicite

            # Ajouter le texte concaténé à la liste
            result_list.append(concatenated_text.strip('·'))  # Retirer le dernier séparateur inutile
        except Exception as e:
            # En cas d'erreur, ajouter une valeur par défaut
            result_list.append('NA')
    return result_list

In [4]:
from selenium.webdriver.common.by import By
import pandas as pd
# Fonction pour extraire les informations de entreprise_elements sur une page
def extract_entreprise_elements(driver, df_new_column):
    entreprise_elements = driver.find_elements(By.CLASS_NAME, 'styles_businessUnitMain__PuwB7')
    informations_entreprises = []

    for i, entreprise in enumerate(entreprise_elements):
        try:
            # Récupérer emplacement (ville et pays)
            emplacement = entreprise.find_element(By.CSS_SELECTOR, 'span.styles_location__ILZb0').text
            location_parts = emplacement.split(",")
            town = location_parts[0].strip() if len(location_parts) > 0 else 'NA'
            country = location_parts[1].strip() if len(location_parts) > 1 else 'NA'
        except Exception:
            town, country = 'NA', 'NA'

        try:
            # Récupérer le nom de l'entreprise
            nom_entreprise = entreprise.find_element(By.CSS_SELECTOR, 'p.typography_heading-xs__jSwUz').text
        except Exception:
            nom_entreprise = 'NA'

        try:
            # Récupérer le trust score
            trust_score_element = entreprise.find_element(By.CSS_SELECTOR, 'span.typography_body-m__xgxZ_')
            trust_score = trust_score_element.text.split()[-1]
            # Valider que le trust_score est un nombre
            trust_score = trust_score if trust_score.replace('.', '', 1).isdigit() else 'NA'
        except Exception:
            trust_score = 'NA'

        try:
            # Récupérer le nombre de reviews
            reviews_text  = entreprise.find_element(By.CSS_SELECTOR, 'p.styles_ratingText__yQ5S7').text.split('|')[-1].strip()
            reviews = ''.join(filter(str.isdigit, reviews_text))
        except Exception:
            reviews = 'NA'

        # Ajouter les informations dans la liste, y compris institution_type extrait précédemment
        informations_entreprises.append({
            'town': town,
            'country': country,
            'institution_type': df_new_column['institution_type'][i] if i < len(df_new_column) else 'NA',
            'company_name': nom_entreprise,
            'trust_score': trust_score,
            'review': reviews
        })
    return informations_entreprises




In [5]:
# Parcourir plusieurs pages
base_url = "https://www.trustpilot.com/categories/atm?page={}"
all_informations_entreprises = []

for page_num in range(1, 3):  # Parcourir les pages 1 et 2 (ajustez selon besoin)
    driver.get(base_url.format(page_num))

    # Étape 1 : Extraction à partir de div_elements
    result_list = extract_div_elements(driver)

    # Créer un DataFrame temporaire pour institution_type
    df_new_column = pd.DataFrame(result_list, columns=['infos'])
    df_new_column['institution_type'] = df_new_column['infos'].apply(lambda x: x.split('·')[0].strip() if '·' in x else 'NA')

    # Étape 2 : Extraction à partir de entreprise_elements
    page_data = extract_entreprise_elements(driver, df_new_column)
    all_informations_entreprises.extend(page_data)

# Créer un DataFrame pandas avec toutes les données
df_entreprises = pd.DataFrame(all_informations_entreprises)


In [6]:
df_entreprises.head(30)

,town,country,institution_type,company_name,trust_score,review
0,Portland,United States,Credit Union,Evergreen Credit Union,4.8,347
1,Lincoln,United States,Car Finance and Loan Company,Liberty First Credit Union,4.8,326
2,Chicago,United States,Money Transfer Service,Crypto Dispensers,4.1,241
3,Cleveland,United States,Beauty Products Vending Machine,VapeTM,4.6,21
4,Atlanta,United States,Bank,Anytime Capital,4.4,15
5,San Francisco,United States,Financial Institution,Swadesh,4.4,10
6,San Diego,United States,Business to Business Service,The Payment HQ,4.0,4
7,San Jose,United States,Debt Relief Service,Meriwest Credit Union,3.8,9
8,Las Vegas,United States,ATM,Coinhub Bitcoin ATM,3.7,8
9,NA,NA,NA,ASDFTEST,3.7,1


In [6]:
# Effectuer une jointure entre les deux DataFrames sur la colonne "company_name" et "Commentaire"
df_final = pd.merge(
    df_entreprises,
    df_details[['Commentaire', 'five_Star_Percentage']],  # Garder uniquement les colonnes nécessaires
    left_on='company_name',  # Colonne dans df_entreprises
    right_on='Commentaire',  # Colonne dans df_details
    how='left'  # Faire une jointure à gauche pour conserver toutes les entreprises
)

# Supprimer la colonne redondante "Commentaire" après la jointure
df_final.drop(columns=['Commentaire'], inplace=True)
#df_final.drop(columns=['five_star_percentage'], inplace=True)




In [7]:
# Afficher les premières lignes pour vérification
df_final.head(30)

,town,country,institution_type,company_name,trust_score,review,five_Star_Percentage
0,Portland,United States,ATM,Evergreen Credit Union,4.8,347,95.0
1,Lincoln,United States,Financial Institution,Liberty First Credit Union,4.8,326,92.0
2,Chicago,United States,Money Transfer Service,Crypto Dispensers,4.1,241,65.0
3,Cleveland,United States,Beauty Products Vending Machine,VapeTM,4.6,21,100.0
4,Atlanta,United States,Bank,Anytime Capital,4.4,15,93.0
5,San Francisco,United States,Financial Institution,Swadesh,4.4,10,100.0
6,San Diego,United States,Business to Business Service,The Payment HQ,4.0,4,100.0
7,San Jose,United States,Debt Relief Service,Meriwest Credit Union,3.8,9,67.0
8,Las Vegas,United States,ATM,Coinhub Bitcoin ATM,3.7,8,37.0
9,NA,NA,NA,ASDFTEST,3.7,1,100.0


In [ ]:
df_new_column

In [5]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

def scrape_trustpilot_comments():
    # Configuration du WebDriver
    driver = webdriver.Chrome()
    url = "https://www.trustpilot.com/categories/atm"
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    time.sleep(2)

    # Gestion du bouton de consentement
    try:
        consent_button = wait.until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler'))
        )
        consent_button.click()
    except Exception as e:
        print("Consent button not found or could not be clicked:", e)



In [ ]:
# Reccuperons les differents liens
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
)
liens = []

next_page_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
next_page_link = next_page_button.get_attribute('href')

driver.execute_script("window.open('{}', '_blank');".format(next_page_link))
driver.switch_to.window(driver.window_handles[-1])

elements_1 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[4]/a")
for element in elements_1:
    href = element.get_attribute('href')
    liens.append(href)
elements_2 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[5]/a")
for element in elements_2:
    href = element.get_attribute('href')
    liens.append(href)

driver.close()
driver.switch_to.window(driver.window_handles[0])

for element in elements:
    link = element.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')
    liens.append(href)

# Attendre que tous les éléments avec la classe spécifique soient présents dans le DOM
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
)

commentaires_par_agence = []

for element in elements:
    link = element.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')

    driver.execute_script("window.open('{}', '_blank');".format(href))

    driver.switch_to.window(driver.window_handles[-1])
    
    
    titre = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/main/div/div[3]/div[2]/div/div/div/section/div[1]/div[2]/h1/span[1]'))
        )
    titre_text = titre.text.strip()

    comments = driver.find_elements(By.CSS_SELECTOR, '.styles_reviewCardInner__EwDq2')
        
    if comments:
        i = 0
        for comment in comments:
            try:
                name_element = driver.find_elements(By.CSS_SELECTOR, 'span[data-consumer-name-typography]')
                name_element = name_element[i]
                user_name = name_element.text
            except Exception as e:
                user_name = 'NA' 
            try:
                location_element = driver.find_elements(By.CSS_SELECTOR, '[data-consumer-country-typography] > span')
                location_element = location_element[i]
                user_location = location_element.text
            except Exception as e:
                user_location = 'NA' 
            try:
                review_content_element = driver.find_element(By.CSS_SELECTOR, 'div[data-review-content]')
                review_content = review_content_element.text
            except Exception as e:
                review_content = 'NA' 
            try:
                title = comment.find_element(By.CSS_SELECTOR, '.typography_heading-s__f7029').text
            except Exception as e:
                title = 'NA'          
            try:
                review_text = comment.find_element(By.CSS_SELECTOR, '.typography_body-l__KUYFJ').text
            except Exception as e:
                review_text = 'NA'          
            try:
                review_count_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.styles_consumerExtraDetails__fxS4S > span.typography_body-m__xgxZ_'))
                )
                review_count = review_count_element.text
            except NoSuchElementException:
                review_count = 'NA'       
            try:
                date_of_experience_element = comment.find_element(By.CSS_SELECTOR, 'p[data-service-review-date-of-experience-typography]')
                date_of_experience = date_of_experience_element.text.replace('Date of experience: ', '').strip()
            except NoSuchElementException:
                date_of_experience = 'NA'                
            try:
                reply_element = comment.find_element(By.CSS_SELECTOR, '.paper_paper__1PY90')
                reply_text = reply_element.find_element(By.CSS_SELECTOR, '.styles_message__shHhX').text
                reply_date_element = reply_element.find_element(By.CSS_SELECTOR, '.styles_replyDate__Iem0_')
                reply_date = reply_date_element.get_attribute('title')
                
                comment_data = {
                'company_name': titre_text,
                'User': user_name,
                'localisation': user_location,
                'Titre': title,
                'commentaire': review_text,
                'nombre_reviews': review_count,
                'date_experience': date_of_experience,
                'reply': {
                    'reply_text': reply_text,
                    'reply_date': reply_date
                }
            }
            except NoSuchElementException:
                comment_data = {
                    'company_name': titre_text,
                    'User': user_name,
                    'localisation': user_location,
                    'Titre': title,
                    'commentaire': review_text,
                    'nombre_reviews': review_count,
                    'date_experience': date_of_experience,
                    'reply': None
                }

            # Ajouter les données du commentaire (avec ou sans réponse) à la liste des commentaires de l'agence
            commentaires_par_agence.append(comment_data)
            i = i+1
            
    driver.close()

    driver.switch_to.window(driver.window_handles[0])


In [9]:

commentaires_par_agence

[{'company_name': 'Evergreen Credit Union',
  'User': 'Lorrie Rand',
  'localisation': 'US',
  'Titre': 'I have all of my banking with…',
  'commentaire': 'I have all of my banking with Evergreen. My car loan included. So when it came time to get a new one I really wanted to keep financing with Evergreen for my new car loan. Everyone was great and helpful and made it work. Thank you again.',
  'nombre_reviews': '1 review',
  'date_experience': 'November 04, 2024',
  'reply': None},
 {'company_name': 'Evergreen Credit Union',
  'User': 'Tony Field',
  'localisation': 'US',
  'Titre': 'Great customer service',
  'commentaire': "Casey was a pleasure to deal with, quick response times, knowledgeable, I've been doing business with Evergreen C.U. many years, always great rates !",
  'nombre_reviews': '1 review',
  'date_experience': 'November 05, 2024',
  'reply': None},
 {'company_name': 'Evergreen Credit Union',
  'User': 'coRpSE',
  'localisation': 'US',
  'Titre': 'bank and staff great, 

In [11]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


def scrape_trustpilot_comments():
    """
    Fonction pour scraper les commentaires d'entreprises depuis Trustpilot.
    
    Returns:
        pd.DataFrame: DataFrame contenant les commentaires extraits avec leurs métadonnées.
    """
    # Configuration du WebDriver
    driver = webdriver.Chrome()
    url = "https://www.trustpilot.com/categories/atm"
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    time.sleep(2)

    # Gestion du bouton de consentement
    try:
        consent_button = wait.until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler'))
        )
        consent_button.click()
    except Exception as e:
        print("Consent button not found or could not be clicked:", e)

    # Collecte des liens d'entreprises
    liens = []
    elements = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
    )
    for element in elements:
        try:
            link = element.find_element(By.TAG_NAME, 'a')
            href = link.get_attribute('href')
            liens.append(href)
        except Exception as e:
            print(f"Error extracting company link: {e}")

    # Naviguer vers la page suivante et récupérer les liens supplémentaires
    try:
        next_page_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
        next_page_link = next_page_button.get_attribute('href')

        driver.execute_script(f"window.open('{next_page_link}', '_blank');")
        driver.switch_to.window(driver.window_handles[-1])

        elements_1 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[4]/a")
        for element in elements_1:
            href = element.get_attribute('href')
            liens.append(href)

        elements_2 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[5]/a")
        for element in elements_2:
            href = element.get_attribute('href')
            liens.append(href)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    except Exception as e:
        print(f"Error navigating to the next page: {e}")

    # Extraction des commentaires pour chaque lien
    commentaires_par_agence = []

    for link in liens:
        try:
            driver.execute_script(f"window.open('{link}', '_blank');")
            driver.switch_to.window(driver.window_handles[-1])

            # Extraire le titre de l'entreprise
            try:
                titre = wait.until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/main/div/div[3]/div[2]/div/div/div/section/div[1]/div[2]/h1/span[1]'))
                )
                titre_text = titre.text.strip()
            except NoSuchElementException:
                titre_text = 'NA'

            # Localiser et extraire les commentaires
            comments = driver.find_elements(By.CSS_SELECTOR, '.styles_reviewCardInner__EwDq2')

            for i, comment in enumerate(comments):
                try:
                    user_name = comment.find_element(By.CSS_SELECTOR, 'span[data-consumer-name-typography]').text
                except NoSuchElementException:
                    user_name = 'NA'

                try:
                    user_location = comment.find_element(By.CSS_SELECTOR, '[data-consumer-country-typography] > span').text
                except NoSuchElementException:
                    user_location = 'NA'

                try:
                    review_text = comment.find_element(By.CSS_SELECTOR, '.typography_body-l__KUYFJ').text
                except NoSuchElementException:
                    review_text = 'NA'

                try:
                    review_date = comment.find_element(By.CSS_SELECTOR, 'p[data-service-review-date-of-experience-typography]').text.replace('Date of experience: ', '').strip()
                except NoSuchElementException:
                    review_date = 'NA'

                commentaires_par_agence.append({
                    'company_name': titre_text,
                    'user_name': user_name,
                    'user_location': user_location,
                    'review_text': review_text,
                    'review_date': review_date,
                })

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(f"Error extracting comments from link {link}: {e}")
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

    # Convertir les commentaires en DataFrame
    df_commentaires = pd.DataFrame(commentaires_par_agence)

    # Fermer le driver
    driver.quit()

    return df_commentaires



In [15]:
import warnings
import os
# Exemple d'exécution
if __name__ == "__main__":
    commentaires_df = scrape_trustpilot_comments()
    #print(commentaires_df.head())
        # Assurez-vous que le répertoire parent 'data' existe
    #data_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', 'data'))
    #os.makedirs(data_dir, exist_ok=True)
    # Utiliser le répertoire de travail actuel pour Jupyter Notebook
    data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
    os.makedirs(data_dir, exist_ok=True)


    # Sauvegarder les commentaires en JSON
    with open(os.path.join(data_dir, 'df_commentaires_par_entreprise.json'), 'w', encoding='utf-8') as f:
        json.dump(commentaires_par_agence, f, ensure_ascii=False, indent=4)



In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_trustpilot_enterprises_details(driver, base_url, num_pages=1):
    """
    Scrape les détails des entreprises Trustpilot, y compris les liens, les scores 5 étoiles et les commentaires.
    
    Arguments :
    - driver : instance du WebDriver Selenium.
    - base_url : URL de base de la catégorie Trustpilot.
    - num_pages : nombre de pages à parcourir.

    Retourne :
    - df_details : DataFrame contenant les détails des entreprises.
    """
    
    liens = []
    score_etoile = []
    commentaires_par_agence = []

    for page_num in range(1, num_pages + 1):
        driver.get(base_url.format(page_num))

        # Étape 1 : Extraire les liens des entreprises
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
        )
        for element in elements:
            try:
                link = element.find_element(By.TAG_NAME, 'a')
                href = link.get_attribute('href')
                liens.append(href)
            except Exception as e:
                print(f"Erreur lors de l'extraction du lien : {e}")
                liens.append('NA')

        # Naviguer vers la page suivante
        try:
            next_page_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
            next_page_link = next_page_button.get_attribute('href')

            driver.execute_script(f"window.open('{next_page_link}', '_blank');")
            driver.switch_to.window(driver.window_handles[-1])

            elements_1 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[4]/a")
            for element in elements_1:
                liens.append(element.get_attribute('href'))

            elements_2 = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/main/div/div[2]/div/section/div[5]/a")
            for element in elements_2:
                liens.append(element.get_attribute('href'))

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception:
            print("Pas de bouton de page suivante trouvé.")

    # Étape 2 : Extraire les détails des entreprises pour chaque lien
    for lien in liens:
        try:
            driver.execute_script(f"window.open('{lien}', '_blank');")
            driver.switch_to.window(driver.window_handles[-1])

            try:
                score_5_etoile_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/main/div/div[4]/section/div[2]/div[2]/label[1]/p[2]')
                score_5_etoile = score_5_etoile_element.text.replace('%', '').strip()
                score_etoile.append(float(score_5_etoile) if score_5_etoile.isdigit() else 'NA')
            except Exception:
                score_etoile.append('NA')

            try:
                titre = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/main/div/div[3]/div[2]/div/div/div/section/div[1]/div[2]/h1/span[1]'))
                )
                titre_text = titre.text.strip()
                commentaires_par_agence.append(titre_text)
            except Exception:
                commentaires_par_agence.append('NA')

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Erreur lors de l'extraction des détails pour le lien : {lien}. Erreur : {e}")
            score_etoile.append('NA')
            commentaires_par_agence.append('NA')

    # Étape 3 : Construire le DataFrame
    data = {
        'Lien': liens,
        'five_Star_Percentage': score_etoile,
        'Commentaire': commentaires_par_agence
    }
    df_details = pd.DataFrame(data)

    return df_details




In [2]:
# Exemple d'utilisation
if __name__ == "__main__":
    from selenium import webdriver

    # Configurer le WebDriver
    driver = webdriver.Chrome()
    base_url = "https://www.trustpilot.com/categories/atm?page={}"
        # Gestion du bouton de consentement
    try:
        consent_button = wait.until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler'))
        )
        consent_button.click()
    except Exception as e:
        print("Consent button not found or could not be clicked:", e)

    try:
        # Appeler la fonction pour scraper les données
        df_details = scrape_trustpilot_enterprises_details(driver, base_url, num_pages=2)

    finally:
        driver.quit()



Consent button not found or could not be clicked: name 'wait' is not defined


In [4]:
print(df_details.head())

                                                Lien  five_Star_Percentage  \
0         https://www.trustpilot.com/review/egcu.org                  95.0   
1  https://www.trustpilot.com/review/libertyfirst...                  92.0   
2  https://www.trustpilot.com/review/cryptodispen...                  65.0   
3       https://www.trustpilot.com/review/vapetm.com                 100.0   
4  https://www.trustpilot.com/review/anytime-capi...                  93.0   

                  Commentaire  
0      Evergreen Credit Union  
1  Liberty First Credit Union  
2           Crypto Dispensers  
3                      VapeTM  
4             Anytime Capital  


In [12]:
from selenium.webdriver.common.by import By
import pandas as pd

# Fonction pour extraire les informations de div_elements sur une page
def extract_div_elements(driver):
    div_elements = driver.find_elements(By.XPATH, '//div[@class="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"]')
    result_list = []  # Liste pour stocker les résultats

    for div in div_elements:
        try:
            # Localiser les sous-éléments dans chaque carte
            elements = div.find_element(By.CLASS_NAME, 'styles_footer__DoJVj')
            elements_2 = elements.find_element(By.CLASS_NAME, 'styles_wrapper___E6__')
            span_elements = elements_2.find_elements(By.TAG_NAME, 'span')

            # Concaténer le texte de tous les <span>
            concatenated_text = ''
            for span in span_elements:
                concatenated_text += span.text + '·'  # Ajout du séparateur explicite

            # Ajouter le texte concaténé à la liste
            result_list.append(concatenated_text.strip('·'))  # Retirer le dernier séparateur inutile
        except Exception as e:
            # En cas d'erreur, ajouter une valeur par défaut
            result_list.append('NA')
    return result_list

In [14]:
from selenium.webdriver.common.by import By
import pandas as pd
# Fonction pour extraire les informations de entreprise_elements sur une page
def extract_entreprise_elements(driver, df_new_column):
    entreprise_elements = driver.find_elements(By.CLASS_NAME, 'styles_businessUnitMain__PuwB7')
    informations_entreprises = []

    for i, entreprise in enumerate(entreprise_elements):
        try:
            # Récupérer emplacement (ville et pays)
            emplacement = entreprise.find_element(By.CSS_SELECTOR, 'span.styles_location__ILZb0').text
            location_parts = emplacement.split(",")
            town = location_parts[0].strip() if len(location_parts) > 0 else 'NA'
            country = location_parts[1].strip() if len(location_parts) > 1 else 'NA'
        except Exception:
            town, country = 'NA', 'NA'

        try:
            # Récupérer le nom de l'entreprise
            nom_entreprise = entreprise.find_element(By.CSS_SELECTOR, 'p.typography_heading-xs__jSwUz').text
        except Exception:
            nom_entreprise = 'NA'

        try:
            # Récupérer le trust score
            trust_score_element = entreprise.find_element(By.CSS_SELECTOR, 'span.typography_body-m__xgxZ_')
            trust_score = trust_score_element.text.split()[-1]
            # Valider que le trust_score est un nombre
            trust_score = trust_score if trust_score.replace('.', '', 1).isdigit() else 'NA'
        except Exception:
            trust_score = 'NA'

        try:
            # Récupérer le nombre de reviews
            reviews_text  = entreprise.find_element(By.CSS_SELECTOR, 'p.styles_ratingText__yQ5S7').text.split('|')[-1].strip()
            reviews = ''.join(filter(str.isdigit, reviews_text))
        except Exception:
            reviews = 'NA'

        # Ajouter les informations dans la liste, y compris institution_type extrait précédemment
        informations_entreprises.append({
            'town': town,
            'country': country,
            'institution_type': df_new_column['institution_type'][i] if i < len(df_new_column) else 'NA',
            'company_name': nom_entreprise,
            'trust_score': trust_score,
            'review': reviews
        })
    return informations_entreprises




In [15]:
# Parcourir plusieurs pages
base_url = "https://www.trustpilot.com/categories/atm?page={}"
all_informations_entreprises = []

for page_num in range(1, 3):  # Parcourir les pages 1 et 2 (ajustez selon besoin)
    driver.get(base_url.format(page_num))

    # Étape 1 : Extraction à partir de div_elements
    result_list = extract_div_elements(driver)

    # Créer un DataFrame temporaire pour institution_type
    df_new_column = pd.DataFrame(result_list, columns=['infos'])
    df_new_column['institution_type'] = df_new_column['infos'].apply(lambda x: x.split('·')[0].strip() if '·' in x else 'NA')

    # Étape 2 : Extraction à partir de entreprise_elements
    page_data = extract_entreprise_elements(driver, df_new_column)
    all_informations_entreprises.extend(page_data)

# Créer un DataFrame pandas avec toutes les données
df_entreprises = pd.DataFrame(all_informations_entreprises)


In [ ]:
# Effectuer une jointure entre les deux DataFrames sur la colonne "company_name" et "Commentaire"
df_final = pd.merge(
    df_entreprises,
    df_details[['Commentaire', 'five_Star_Percentage']],  # Garder uniquement les colonnes nécessaires
    left_on='company_name',  # Colonne dans df_entreprises
    right_on='Commentaire',  # Colonne dans df_details
    how='left'  # Faire une jointure à gauche pour conserver toutes les entreprises
)

# Supprimer la colonne redondante "Commentaire" après la jointure
df_final.drop(columns=['Commentaire'], inplace=True)
#df_final.drop(columns=['five_star_percentage'], inplace=True)




In [16]:
# Effectuer une jointure entre les deux DataFrames sur la colonne "company_name" et "Commentaire"
df_final = pd.merge(
    df_entreprises,
    df_details[['Commentaire', 'five_Star_Percentage']],  # Garder uniquement les colonnes nécessaires
    left_on='company_name',  # Colonne dans df_entreprises
    right_on='Commentaire',  # Colonne dans df_details
    how='left'  # Faire une jointure à gauche pour conserver toutes les entreprises
)

# Supprimer la colonne redondante "Commentaire" après la jointure
df_final.drop(columns=['Commentaire'], inplace=True)
#df_final.drop(columns=['five_star_percentage'], inplace=True)




In [17]:
df_final.head(20)

,town,country,institution_type,company_name,trust_score,review,five_Star_Percentage
0,Portland,United States,Credit Union,Evergreen Credit Union,4.8,347,95.0
1,Lincoln,United States,Car Finance and Loan Company,Liberty First Credit Union,4.8,326,92.0
2,Chicago,United States,Money Transfer Service,Crypto Dispensers,4.1,241,65.0
3,Cleveland,United States,Beauty Products Vending Machine,VapeTM,4.6,21,100.0
4,Atlanta,United States,Bank,Anytime Capital,4.4,15,93.0
5,San Francisco,United States,Financial Institution,Swadesh,4.4,10,100.0
6,San Diego,United States,Business to Business Service,The Payment HQ,4.0,4,100.0
7,San Jose,United States,Debt Relief Service,Meriwest Credit Union,3.8,9,67.0
8,Las Vegas,United States,ATM,Coinhub Bitcoin ATM,3.7,8,37.0
9,NA,NA,NA,ASDFTEST,3.7,1,100.0
